In [8]:
import torch
import torch_geometric.utils as tg
from rephine_mt import compute_rephine_batched_mt
from spectre import compute_spectre_batched_mt
from ph_cpu import compute_persistence_homology_batched_mt
import collections
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx


In [9]:
def get_data(name):
    graphs = nx.read_graph6(f"./datasets/cayley/{name}.g6")
    dataset = []
    for i, g in enumerate(graphs):
        pyg_graph = from_networkx(g)
        x = torch.ones(len(g.nodes))
        new_g = Data(x=x.unsqueeze(1), edge_index=pyg_graph.edge_index)
        dataset.append(new_g)
    return dataset

In [10]:
def compute_persistence(diagram_type, x, e, edge_index, vertex_slices, edge_slices):
    filtered_v = x
    filtered_e = e
    if diagram_type == "rephine":
        compute = compute_rephine_batched_mt
    elif diagram_type == "spectre":
        compute = compute_spectre_batched_mt
    elif diagram_type == "standard":
        filtered_e, _ = torch.max(
            torch.stack((filtered_v[edge_index[0]], filtered_v[edge_index[1]])),
            axis=0,
        )
        compute = compute_persistence_homology_batched_mt
    elif diagram_type == "edge":
        compute = compute_rephine_batched_mt

    vertex_slices = vertex_slices.cpu().long()
    edge_slices = edge_slices.cpu().long()
    filtered_v = filtered_v.transpose(1, 0).cpu().contiguous()
    filtered_e = filtered_e.transpose(1, 0).cpu().contiguous()
    edge_index = edge_index.cpu().transpose(1, 0).contiguous()

    
    persistence0, persistence1 = compute(
        filtered_v, filtered_e, edge_index, vertex_slices, edge_slices
    )

    persistence0 = persistence0.to(x.device)
    persistence1 = persistence1.to(x.device)

    if diagram_type == "rephine" or diagram_type == 'spectre' or diagram_type == 'edge':
        full_size = persistence0.shape[2]
        indices = list(range(3, full_size, 1))
        persistence0 = persistence0[:, :, [0, 2, 1] + indices]

        persistence0 = torch.cat(
            (
                torch.zeros((persistence0.shape[0], persistence0.shape[1], 1)).to(
                    x.device
                ),
                persistence0,
            ),
            dim=-1,
        )
        persistence1 = torch.cat(
            (
                torch.zeros((persistence1.shape[0], persistence1.shape[1], 1)).to(
                    x.device
                ),
                persistence1,
            ),
            dim=-1,
        )

        persistence0[persistence0.isnan()] = 1000

    return persistence0.squeeze(), persistence1.squeeze()


In [11]:
def apply_filtering_function(g):
    adj = tg.to_dense_adj(g.edge_index)
    degree = adj.sum(dim=-1).squeeze()
    g.x = degree.unsqueeze(dim=0).T
    correct_idx = g.edge_index[0] <= g.edge_index[1]
    new_edge_index = g.edge_index[:, correct_idx] # remove "duplicated" edges 
    edge_features = torch.zeros(new_edge_index.shape[1], 1)
    for i in range(new_edge_index.shape[1]):
        e = [new_edge_index[0, i].item(), new_edge_index[1, i].item()]
        edge_features[i] = 4 - degree[e[0]] - degree[e[1]]
        neighbors = g.edge_index[1, g.edge_index[0] == e[0]].tolist()
        neighbors2 = g.edge_index[1, g.edge_index[0] == e[1]].tolist()
        edge_features[i] += 3 * len(set(neighbors) & set(neighbors2))
    g.edge_index = new_edge_index 
    g.edge_features = edge_features     
    return g


def compute_diagram(g, diagram):
    vs = torch.tensor([0, g.x.shape[0]]).long()
    es = torch.tensor([0, g.edge_index.shape[1]]).long()
    d0, d1 = compute_persistence(diagram_type=diagram, x=g.x, e=g.edge_features, 
                                   edge_index=g.edge_index, 
                                   vertex_slices=vs, edge_slices=es)
    
    if diagram=='edge':
        d0 = d0[:, :2]
        d1 = d1[:, :2]
    return d0, d1    

def compare_diagrams(dG_0, dG_1, dH_0, dH_1):
    lG = [] 
    for b in range(dG_0.shape[0]):
        lG.append(tuple(dG_0[b].numpy().round(3)))

    lH = []
    for b in range(dH_0.shape[0]):
        lH.append(tuple(dH_0[b].numpy().round(3)))

    lG1 = []
    for b in range(dG_1.shape[0]):
        lG1.append(tuple(dG_1[b].numpy().round(3)))

    lH1 = [] 
    for b in range(dH_1.shape[0]):
        lH1.append(tuple(dH_1[b].numpy().round(3)))
    
    return (collections.Counter(lG) == collections.Counter(lH)) and (collections.Counter(lG1) == collections.Counter(lH1))


In [12]:
choices = [
    "minCayleyGraphs12Vertices", 
    "minCayleyGraphs16Vertices",
    "minCayleyGraphs20Vertices", 
    "minCayleyGraphs24Vertices",
    "minCayleyGraphs32Vertices", 
    "minCayleyGraphs36Vertices",
    "minCayleyGraphs60Vertices", 
    "minCayleyGraphs63Vertices"
]

diagrams = ['standard', 'edge', 'rephine', 'spectre']
for setting in choices:
    dataset = get_data(setting)
    results = {}
    j = 0
    for g in dataset:
        g = apply_filtering_function(g)
        values = dict()
        for diagram in diagrams:
            d_0, d_1 = compute_diagram(g, diagram) 
            values[diagram] = [d_0, d_1]
        results[j]=values
        j = j+1
    
    arr = torch.tensor(list(range(0, len(dataset))))
    indices = torch.combinations(arr, 2)
    
    for diagram in diagrams:
        count = 0
        for i in indices:
            a = i.tolist()
            d10, d11 = results[a[0]][diagram]
            d20, d21 = results[a[1]][diagram]

            if compare_diagrams(d10, d11, d20, d21):
                count += 1
                
        print(f'{setting}, {diagram}:, {(1.0-count/len(indices)):.2f}')

minCayleyGraphs12Vertices, standard:, 0.67
minCayleyGraphs12Vertices, edge:, 0.95
minCayleyGraphs12Vertices, rephine:, 0.95
minCayleyGraphs12Vertices, spectre:, 1.00
minCayleyGraphs16Vertices, standard:, 0.83
minCayleyGraphs16Vertices, edge:, 0.83
minCayleyGraphs16Vertices, rephine:, 0.83
minCayleyGraphs16Vertices, spectre:, 1.00
minCayleyGraphs20Vertices, standard:, 0.61
minCayleyGraphs20Vertices, edge:, 0.61
minCayleyGraphs20Vertices, rephine:, 0.61
minCayleyGraphs20Vertices, spectre:, 1.00
minCayleyGraphs24Vertices, standard:, 0.65
minCayleyGraphs24Vertices, edge:, 0.86
minCayleyGraphs24Vertices, rephine:, 0.86
minCayleyGraphs24Vertices, spectre:, 1.00
minCayleyGraphs32Vertices, standard:, 0.76
minCayleyGraphs32Vertices, edge:, 0.76
minCayleyGraphs32Vertices, rephine:, 0.76
minCayleyGraphs32Vertices, spectre:, 1.00
minCayleyGraphs36Vertices, standard:, 0.69
minCayleyGraphs36Vertices, edge:, 0.84
minCayleyGraphs36Vertices, rephine:, 0.84
minCayleyGraphs36Vertices, spectre:, 1.00
minC